In [8]:
import ee
import ipygee as ui
import datetime
import numpy as np
from tqdm import tqdm

# Function to print start and end time of dataset

In [2]:
def print_edge_time(dataset, start=True):
    if start:
        date = ee.Date(dataset.first().get('system:time_start'))
    else:
        date = ee.Date(dataset.sort('system:time_start', False).first().get('system:time_start'))

    date_dict = date.getInfo()

    print("The number of milliseconds since 1970-01-01T00:00:00Z.: ", date_dict['value'])

    print("Formatted date", date.format('Y-M-d').getInfo())

# Setting up earth engine API

In [3]:
# Trigger the authentication flow.
ee.Authenticate()

# Initialize the library.
ee.Initialize()


Successfully saved authorization token.


# Dataset selection, time period, bands, geometry 

In [4]:
model = 'CNRM-CM5'
scenario = 'rcp45' #to get data after 2006
# var scenario = 'historical' to get data before 2006

bands = ['tasmax', 'tasmin', 'pr']
bands_len = len(bands)

In [5]:
start_y = 1965
start_m = 1

end_y = 2022

start_date = datetime.datetime(year=start_y, month=start_m, day=1)
end_date = datetime.datetime(year=end_y, month=start_m, day=1)

In [6]:
xMin, yMin, xMax, yMax = 30.21928091061925, 40.538579354365346, 46.58915395749425, 49.289072811288115
rectangleBounds = ee.Geometry.Rectangle(
  [xMin, yMin, xMax, yMax]
)

# Casting to numpy array, dumping into dictionary

In [9]:
bands_dict = {}
for j in tqdm(range(len(bands))):
    dataset = ee.ImageCollection('NASA/NEX-GDDP').filterBounds(rectangleBounds).filterMetadata('model', 'equals', model).filterMetadata('scenario', 'equals', scenario).filter(ee.Filter.date(start_date, end_date)).select(bands[j])
    tmp = dataset.toArrayPerBand().sampleRectangle(rectangleBounds).get(bands[j]).getInfo()
    np_band_j = np.array(tmp)
    bands_dict[bands[j]] = np_band_j
    # img = dataset.toBands()
                  

100%|██████████| 3/3 [00:25<00:00,  8.44s/it]


# Double check timing

In [10]:
print("Start")
print_edge_time(dataset=dataset, start=True)
print("End")
print_edge_time(dataset=dataset, start=False)

Start
The number of milliseconds since 1970-01-01T00:00:00Z.:  1136073600000
Formatted date 2006-1-1
End
The number of milliseconds since 1970-01-01T00:00:00Z.:  1640908800000
Formatted date 2021-12-31
